#### I tried to make the code of this Notebook as simple as possible. If you think you learned something from it. Do Upvote. ⬆️ 😊

![](https://wallpaperaccess.com/full/218450.jpg)

## Required Libraries

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
sns.set_style('darkgrid')

from PIL import Image, ImageDraw
import tensorflow as tf

import os
import ast  ## Change str -> list.
import sys
import time

import warnings
warnings.filterwarnings('ignore')

import greatbarrierreef

## Files
#### **train** - Folder containing training set photos of the form video_{video_id}/{video_frame_number}.jpg.

#### **[train/test].csv** - Metadata for the images. As with other test files, most of the test metadata data is only available to your notebook upon submission. Just the first few rows available for download.

> #### **video_id** - ID number of the video the image was part of. The video ids are not meaningfully ordered.
> #### **video_frame** - The frame number of the image within the video. Expect to see occasional gaps in the frame number from when the diver surfaced.
> #### **sequence** - ID of a gap-free subset of a given video. The sequence ids are not meaningfully ordered.
> #### **sequence_frame** - The frame number within a given sequence.
> #### **image_id** - ID code for the image, in the format '{video_id}-{video_frame}'
> #### **annotations** - The bounding boxes of any starfish detections in a string format that can be evaluated directly with Python. Does not use the same format as the predictions you will submit. Not available in test.csv. A bounding box is described by the pixel coordinate (x_min, y_min) of its upper left corner within the image together with its width and height in pixels.
#### **example_sample_submission.csv** - A sample submission file in the correct format. The actual sample submission will be provided by the API; this is only provided to illustrate how to properly format predictions. The submission format is further described on the Evaluation page.

#### **example_test.npy** - Sample data that will be served by the example API.

#### **greatbarrierreef** - The image delivery API that will serve the test set pixel arrays. You may need Python 3.7 and a Linux environment to run the example offline without errors.

## Loading Data

In [ ]:
df_train = pd.read_csv('../input/tensorflow-great-barrier-reef/train.csv')
df_train['img_path'] = os.path.join('../input/tensorflow-great-barrier-reef/train_images')+"/video_"+df_train.video_id.astype(str)+"/"+df_train.video_frame.astype(str)+".jpg"
df_train.head()

## Exploratory Data Analysis

#### Let's count how many images are there from each of the three videos.

In [ ]:
plt.figure(figsize=(8,5))
sns.countplot(df_train['video_id'], color='#2196F3')

#### Now, let's have a look at how many images with bounding boxes are available.

In [ ]:
with_annotation = len(df_train[df_train['annotations'] != '[]'])
without_annotation = len(df_train[df_train['annotations'] == '[]'])

labels = ['Without Bounding Box', 'With Bounding Box']

fig = go.Figure([go.Bar(x=labels, 
                        y=[without_annotation, with_annotation], width=0.6)])

fig.update_layout(autosize=False, width=700, height=400, margin=dict(l=60, r=60, b=50, t=50, pad=4))
fig.show()

#### Clearly, very few number of images have annotations as compared to without annotations images.

#### Now, we'll going to find out how many bouning boxes are there in each annotation columns.
#### In order to do that we can simply count the number of opening curly brackets - '{' in annotation column.
> **Example** :-  `df_train['annotations'][12843]` *where 12843 is a random number.*

> Gives -> 

> "[{'x': 338, 'y': 229, 'width': 45, 'height': 27},
>
> {'x': 357, 'y': 285, 'width': 37, 'height': 38}, 
> 
> {'x': 173, 'y': 588, 'width': 65, 'height': 56}, 
> 
> {'x': 234, 'y': 598, 'width': 31, 'height': 26}]" 

> **We can see that it has 4 bounding box coordinates, the simple way to get this is to count number of opening curly brackets.**

In [ ]:
# creating new column which contains the total number of bounding boxes
df_train['No_bbox'] = df_train['annotations'].apply(lambda x:x.count('{')) 

# Example

n = df_train['No_bbox'][12843]
print(df_train['annotations'][12843])
print(f'Number of bounding boxes are : {n}.')

In [ ]:
fig = px.bar(df_train['No_bbox'].value_counts().drop(0), title='Count of Bounding Boxes')
fig.update_layout(autosize=False, width=700, height=400, margin=dict(l=60, r=60, b=50, t=50, pad=4))
fig.show()

#### Most of the images has only single Bounding Box. Very few have more than 5 BBox.

#### Now, lets change 'annotations' from string to list data type using ***ast***.

In [ ]:
df_train['annotations'] = df_train['annotations'].apply(ast.literal_eval)
df_train.head()

#### Creating new DataFrame which carry images containing more than 1 Bounding Boxes (You could take any value you like) and then use that row's 'video_id' to see image with Bounding Boxes.

In [ ]:
df2 = df_train[df_train['annotations'].astype(str) != "[]"]
df2 = df2[df2['No_bbox'] == 5]
df2.head()

## Data Visualization

In [ ]:
def img_viz(df, id):
    image = df_train['img_path'][id]
    img = Image.open(image)
    
    for box in df_train['annotations'][id]:
        shape = [box['x'], box['y'], box['x']+box['width'], box['y']+box['height']]
        ImageDraw.Draw(img).rectangle(shape, outline ="red", width=3)
    display(img)

In [ ]:
img_viz(df_train, id=5474)

#### Awesome!!!

## Model
#### Load the TensorFlow COTS detection model into memory and define some util functions for running inference.
#### Read more about it [here](https://www.kaggle.com/khanhlvg/cots-detection-w-tensorflow-object-detection-api).

In [ ]:
INPUT_DIR = '../input/tensorflow-great-barrier-reef/'
sys.path.insert(0, INPUT_DIR)

In [ ]:
MODEL_DIR = '../input/cots-detection-w-tensorflow-object-detection-api/cots_efficientdet_d0'
start_time = time.time()
tf.keras.backend.clear_session()
detect_fn_tf_odt = tf.saved_model.load(os.path.join(os.path.join(MODEL_DIR, 'output'), 'saved_model'))
end_time = time.time()
elapsed_time = end_time - start_time
print('Elapsed time: ' + str(elapsed_time) + 's')

#### Helper functions

In [ ]:
def load_image_into_numpy_array(path):
    """Load an image from file into a numpy array.

    Puts image into numpy array to feed into tensorflow graph.
    Note that by convention we put it into a numpy array with shape
    (height, width, channels), where channels=3 for RGB.

    Args:
    path: a file path (this can be local or on colossus)

    Returns:
    uint8 numpy array with shape (img_height, img_width, 3)
    """
    img_data = tf.io.gfile.GFile(path, 'rb').read()
    image = Image.open(io.BytesIO(img_data))
    (im_width, im_height) = image.size
    
    return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

def detect(image_np):
    """Detect COTS from a given numpy image."""

    input_tensor = np.expand_dims(image_np, 0)
    start_time = time.time()
    detections = detect_fn_tf_odt(input_tensor)
    return detections

#### Creating Environment

In [ ]:
env = greatbarrierreef.make_env()   # initialize the environment
iter_test = env.iter_test()    # an iterator which loops over the test set and sample submission

## Prediction | Submission

In [ ]:
DETECTION_THRESHOLD = 0.13

submission_dict = {
    'id': [],
    'prediction_string': [],
}

for (image_np, sample_prediction_df) in iter_test:
    height, width, _ = image_np.shape
    
    # Run object detection using the TensorFlow model.
    detections = detect(image_np)
    
    # Parse the detection result and generate a prediction string.
    num_detections = detections['num_detections'][0].numpy().astype(np.int32)
    predictions = []
    for index in range(num_detections):
        score = detections['detection_scores'][0][index].numpy()
        if score < DETECTION_THRESHOLD:
            continue

        bbox = detections['detection_boxes'][0][index].numpy()
        y_min = int(bbox[0] * height)
        x_min = int(bbox[1] * width)
        y_max = int(bbox[2] * height)
        x_max = int(bbox[3] * width)
        
        bbox_width = x_max - x_min
        bbox_height = y_max - y_min
        
        predictions.append('{:.2f} {} {} {} {}'.format(score, x_min, y_min, bbox_width, bbox_height))
    
    # Generate the submission data.
    prediction_str = ' '.join(predictions)
    sample_prediction_df['annotations'] = prediction_str
    env.predict(sample_prediction_df)

    print('Prediction:', prediction_str)

## References
* [COTS detection w/ TensorFlow Object Detection API](https://www.kaggle.com/khanhlvg/cots-detection-w-tensorflow-object-detection-api)
* [EDA: Let's understand the data - protect the reef](https://www.kaggle.com/casfranco/eda-let-s-understand-the-data-protect-the-reef)
* [Inference using EfficientDet-D0 model](https://www.kaggle.com/khanhlvg/inference-using-efficientdet-d0-model-tensorflow/notebook)